In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import solve_ivp

# Define parameters
L = 1.0       # Length of the rod
Nx = 50       # Number of spatial points
dx = L / Nx   # Space step
T = 0.2       # Total time
Nt = 201     # Number of time steps
dt = T / Nt   # Time step
alpha = 0.1  # Thermal diffusivity

# Stability condition (for explicit scheme)
s = alpha * dt / dx**2
if s > 0.5:
    print("Warning: Stability condition violated! Reduce dt or increase dx.")

# Initial condition: Gaussian pulse
x = np.linspace(0, L, Nx)
u0 = np.exp(-100 * (x - L/2)**2)  # Gaussian initial profile

# Boundary conditions (Dirichlet)
u0[0] = 0
u0[-1] = 0

# Function for the right-hand side of the PDE
def heat_eq(t, u):
    u = u.reshape((Nx,))
    du_dt = np.zeros_like(u)
    for i in range(1, Nx - 1):
        du_dt[i] = alpha * (u[i-1] - 2*u[i] + u[i+1]) / dx**2
    return du_dt

# Solve the PDE using solve_ivp
solution = solve_ivp(heat_eq, [0, T], u0, t_eval=np.linspace(0, T, Nt), method='RK45')

# Reshape the solution to match the time and space dimensions
u = solution.y


In [2]:
u.shape

(50, 201)

In [3]:
from scipy.stats import skewnorm

noise = skewnorm.rvs(a=1, scale=0.3, size=u.shape)
u += noise

In [4]:
import matplotlib.pyplot as plt
import numpy as np
from matplotlib import animation
from tqdm import tqdm
import h5py
def visualize_burgers(xcrd, data, path):
    """
    This function animates the Burgers equation

    Args:
    path : path to the desired file
    param: PDE parameter of the data shard to be visualized
    """
    fig, ax = plt.subplots()
    ims = []

    for i in tqdm(range(data.shape[0])):
        if i == 0:
            im = ax.plot(xcrd, data[i].squeeze(), animated=True, color="blue")
        else:
            im = ax.plot(xcrd, data[i].squeeze(), animated=True, color="blue")
        ims.append([im[0]])

    ani = animation.ArtistAnimation(fig, ims, interval=50, blit=True, repeat_delay=1000)

    writer = animation.PillowWriter(fps=15, bitrate=1800)
    ani.save(path, writer=writer)
    plt.close(fig)
visualize_burgers(x, u.T, "a.gif")

100%|██████████| 201/201 [00:00<00:00, 3411.91it/s]


In [6]:
import torch
import torch.nn as nn
import numpy as np
from scipy.io import loadmat
from tqdm import tqdm


class PINN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(PINN, self).__init__()
        self.layers = nn.ModuleList(
            [
                (
                    nn.Linear(input_size if i == 0 else hidden_size, hidden_size)
                    if i % 2 == 0
                    else nn.Tanh()
                )
                for i in range(20)
            ]
        )
        self.layers.append(nn.Linear(hidden_size, output_size))
        self.loss = nn.MSELoss()
        self.alpha = nn.Parameter(torch.tensor([0.1], requires_grad=True).to("cuda"))
        self.optimizer = torch.optim.Adam(self.parameters(), lr=1e-3)

    def forward(self, x):
        for layer in self.layers:
            x = layer(x)
        return x

    def loss_fn(self, x, u):
        u_pred = self.forward(x)
        return self.loss(u_pred, u)

    def residual_loss(self, xtrain, fhat):
        x = xtrain[:, 0]
        t = xtrain[:, 1]
        g = xtrain.clone()
        g.requires_grad = True
        u_pred = self.forward(g)
        u_x_t = torch.autograd.grad(
            u_pred, g, torch.ones_like(u_pred), create_graph=True
        )[0]
        u_x, u_t = u_x_t[:, 0], u_x_t[:, 1]
        u_xx = torch.autograd.grad(u_x, g, torch.ones_like(u_x), create_graph=True)[0][
            :, 0
        ]
        residual = u_t - self.alpha * u_xx
        return self.loss(residual, fhat)

    def total_loss(self, xtrain, utrain, fhat):
        return self.loss_fn(xtrain, utrain) + self.residual_loss(xtrain, fhat)

    def train_model(self, xtrain, utrain, epochs=1000):
        fhat = torch.zeros(xtrain.shape[0], 1, device="cuda")
        for epoch in tqdm(range(epochs)):
            self.optimizer.zero_grad()
            loss = self.total_loss(xtrain, utrain, fhat)
            loss.backward()
            self.optimizer.step()
            if epoch % 1000 == 0:
                print(f"Epoch {epoch}, Loss {loss.item()}, alpha {self.alpha.item()}")



model = PINN(input_size=2, hidden_size=20, output_size=1).to("cuda")

In [7]:
t = np.linspace(0, 0.2, 201)
X, T = np.meshgrid(x, t)
u = u.T
X.shape, T.shape, u.shape

((201, 50), (201, 50), (201, 50))

In [8]:
xtrue = np.hstack((X.flatten()[:, None], T.flatten()[:, None]))
idx = np.random.choice(201*50, 5000, replace=False)
xtrain = xtrue[idx, :]
utrain = u.flatten()[idx][:, None]
xtrain.shape, utrain.shape

((5000, 2), (5000, 1))

In [9]:
device = torch.device("cuda")
Xtrain = torch.tensor(xtrain, dtype=torch.float32).to(device)
Xtrue = torch.tensor(xtrue, dtype=torch.float32).to(device)
Utrain = torch.tensor(utrain, dtype=torch.float32).to(device)
utrue = u.flatten()[:, None]
utrue = torch.tensor(utrue, dtype=torch.float32).to(device)
Xtrain.shape, Xtrue.shape, Utrain.shape, utrue.shape

(torch.Size([5000, 2]),
 torch.Size([10050, 2]),
 torch.Size([5000, 1]),
 torch.Size([10050, 1]))

In [10]:
model = PINN(input_size=2, hidden_size=20, output_size=1).to("cuda")
model.train_model(Xtrain, Utrain, epochs=10000)

  0%|          | 0/10000 [00:00<?, ?it/s]/home/shusrith/projects/torch/lib/python3.12/site-packages/torch/autograd/graph.py:825: UserWarning: Attempting to run cuBLAS, but there was no current CUDA context! Attempting to set the primary context... (Triggered internally at ../aten/src/ATen/cuda/CublasHandlePool.cpp:135.)
  return Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass
/home/shusrith/projects/torch/lib/python3.12/site-packages/torch/nn/modules/loss.py:608: UserWarning: Using a target size (torch.Size([5000, 1])) that is different to the input size (torch.Size([5000])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
  0%|          | 15/10000 [00:00<02:26, 68.37it/s]

Epoch 0, Loss 0.1652015894651413, alpha 0.10082937777042389


 10%|█         | 1028/10000 [00:07<01:04, 139.14it/s]

Epoch 1000, Loss 0.0666489526629448, alpha 0.0015933114336803555


 20%|██        | 2030/10000 [00:14<00:56, 140.44it/s]

...

 80%|████████  | 8018/10000 [00:57<00:14, 139.03it/s]

Epoch 8000, Loss 0.06305033713579178, alpha 0.08236825466156006


 90%|█████████ | 9026/10000 [01:04<00:07, 138.47it/s]

Epoch 9000, Loss 0.06319021433591843, alpha 0.0839734748005867


100%|██████████| 10000/10000 [01:11<00:00, 138.98it/s]


In [11]:
model.alpha.item()

0.08536027371883392